Install packages

In [ ]:
pip install -q langchain
pip install -q langchain-community
pip install -q langchain-chroma
pip install -q langchain-openai

Import packages

In [ ]:
import os
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = ""

Initiate model

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini")

Hello World with your chatbot!

In [ ]:
resp1 = model.invoke([HumanMessage(content="Hi! I'm Bob")])
print(resp1.content)

Does it remember you?

In [ ]:
resp2 = model.invoke([HumanMessage(content="What's my name?")])
print(resp2.content)

Let's give it some memory!

In [ ]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

store = {}
model_with_history = RunnableWithMessageHistory(model, get_session_history)

Tell it your name again...

In [ ]:
response = model_with_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config={"configurable": {"session_id": "test1"}},
)

print(response.content)

...now it remembers!

In [ ]:
response = model_with_history.invoke(
    [HumanMessage(content="What's my name?")],
    config={"configurable": {"session_id": "test1"}},
)

print(response.content)

Manage multiple chat histories at once by changing session_id

In [ ]:
response = model_with_history.invoke(
    [HumanMessage(content="What's my name?")],
    config={"configurable": {"session_id": "test2"}},
)

print(response.content)

Let's inspect the history to see what's being passed in the prompt

In [ ]:
for message in store["test1"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"
    print(f"{prefix}: {message.content}\n")